# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import requests

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

weather = pd.read_csv("/Users/emilyye/BOOTCAMP WORK/upenn-phi-virt-data-09-2021-u-c/06-Python-APIs/Homework/Instructions/weather_df_clean.csv")

In [3]:
weather.head()

,Unnamed: 0,City,Country,Date,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,busselton,AU,1635620025,-33.6500,115.3333,44.04,90,75,6.29
1,1,svetlyy,RU,1635620026,54.6750,20.1347,50.05,61,4,13.24
2,3,cervo,ES,1635620026,43.6702,-7.4101,63.19,79,9,6.93
3,4,ushuaia,AR,1635619890,-54.8000,-68.3000,49.66,53,0,18.41
4,5,rikitea,PF,1635619941,-23.1203,-134.9692,74.97,85,100,20.74


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
hum_map = gmaps.figure()

coord = weather[["Latitude","Longtitude"]]

heat_layer = gmaps.heatmap_layer(coord, weights = weather["Humidity"], dissipating=False, max_intensity=300, point_radius=5)

hum_map.add_layer(heat_layer)

hum_map


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# creating a df that contains perfect weather
weather_perf = weather.loc[(weather["Cloudiness"] ==0)&
                           (weather["Wind Speed"] < 10)&
                           (weather["Max Temperature"]<70)&
                           (weather["Max Temperature"]>60)]
weather_perf_clean = weather_perf.dropna(how = "any")
weather_perf_clean

,Unnamed: 0,City,Country,Date,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed
23,25,cape town,ZA,1635619881,-33.9258,18.4232,63.10,80,0,3.00
134,149,christchurch,NZ,1635619794,-43.5333,172.6333,61.92,44,0,4.00
142,159,tura,IN,1635620065,25.5198,90.2201,64.56,68,0,2.62
228,258,naze,JP,1635620091,28.3667,129.4833,67.37,78,0,6.62
231,261,san quintin,MX,1635620092,30.4833,-115.9500,69.93,67,0,9.57
332,372,ancud,CL,1635620125,-41.8697,-73.8203,60.33,46,0,2.17
529,583,brindisi,IT,1635620185,40.5833,17.6667,61.77,68,0,2.86


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_perf_clean

hotel_name_list = []


In [7]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():

    lat = row["Latitude"]
    long = row["Longtitude"]
    coordinates = str(lat)+","+str(long)
    params = {
        "location": f"{lat},{long}",
        "radius": 5000,
        "keyword": "hotel",
        "type": "hotel",
        "key": g_key,
    }

    hotel_response = requests.get(url, params=params).json()
    try:
        hotel_name = hotel_response["results"][0]["name"]
        print(hotel_name)
        hotel_name_list.append(hotel_name)
    except:
        print("No hotel within radius")
        hotel_name_list.append("")


President Hotel
Heartland Hotel Cotswold
Hotel C&C
山羊島ホテル
Old Mill Hotel
Panamericana Hotel Ancud
B&B Dimora Baronia


In [8]:
hotel_df["Hotel Name"] = hotel_name_list

hotel_df

,Unnamed: 0,City,Country,Date,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
23,25,cape town,ZA,1635619881,-33.9258,18.4232,63.10,80,0,3.00,President Hotel
134,149,christchurch,NZ,1635619794,-43.5333,172.6333,61.92,44,0,4.00,Heartland Hotel Cotswold
142,159,tura,IN,1635620065,25.5198,90.2201,64.56,68,0,2.62,Hotel C&C
228,258,naze,JP,1635620091,28.3667,129.4833,67.37,78,0,6.62,山羊島ホテル
231,261,san quintin,MX,1635620092,30.4833,-115.9500,69.93,67,0,9.57,Old Mill Hotel
332,372,ancud,CL,1635620125,-41.8697,-73.8203,60.33,46,0,2.17,Panamericana Hotel Ancud
529,583,brindisi,IT,1635620185,40.5833,17.6667,61.77,68,0,2.86,B&B Dimora Baronia


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longtitude"]]

In [10]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
hum_map.add_layer(markers)
hum_map

Figure(layout=FigureLayout(height='420px'))